In [31]:
import cv2
from facenet_pytorch import MTCNN
import face_recognition

from pytube import YouTube
import cv2
import numpy as np
from keras.models import load_model

class VideoProcessor:
    def __init__(self, batch_size=50, skip_frames=1):
        self.batch_size = batch_size
        self.skip_frames = skip_frames
        self.frames_without_faces = 0

    def load_video_from_youtube(self, yt_link):
        self.yt_link = yt_link
        # Download the YouTube video and get the highest resolution stream
        yt_video = YouTube(self.yt_link)
        stream = yt_video.streams.get_highest_resolution()

        # Open the video stream using OpenCV
        self.video = cv2.VideoCapture(stream.url)

        # Get available and used resolution - Debugging
        #self.available_resolutions = [streams.resolution for streams in yt_video.streams.filter(type="video", progressive=True)]
        #self.used_resolution = stream.resolution

        # Get the number of frames in the video
        self.frame_count = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))

        # Get the frame rate of the video
        self.fps = int(self.video.get(cv2.CAP_PROP_FPS))

        # Get the height and width of the video frames
        self.height = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.width = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))

        # Calculate the total number of frames to process after skipping frames
        self.total_frames = len(range(0, self.frame_count, self.skip_frames))

        # Calculate the number of batches required to process all the frames
        self.num_batches = int(np.ceil(self.total_frames / self.batch_size))

    def get_batches(self):
        # Initialize an empty numpy array to hold the frames
        frames = np.empty((self.batch_size, self.height, self.width, 3), np.dtype('uint8'))

        self.frames_read = 0

        # Read the frames in batches and fill up the numpy array
        for batch_start in range(0, self.frame_count, self.batch_size * self.skip_frames):
            batch_end = min(batch_start + (self.batch_size * self.skip_frames), self.frame_count)
            batch_index = 0

            for i in range(batch_start, batch_end):
                ret, frame = self.video.read()
                self.frames_read +=1
                if not ret:
                    break

                if i % self.skip_frames == 0:
                    frames[batch_index] = frame
                    batch_index += 1

            # Resize the numpy array to fit the actual number of frames in the batch
            if batch_index < self.batch_size:
                frames = frames[:batch_index]

            # Yield the current batch of frames
            yield frames

        # Release the video stream
        self.video.release()

class FaceDetector:
    def __init__(self, detection_type="face_recognition"):
        self.detection_type = detection_type

        if self.detection_type == "MTCNN":
            # Initialize the MTCNN face detector
            self.face_detector = MTCNN(keep_all=True, post_process=False, margin=20)
        elif self.detection_type == "haarcascade":
            # Load the Haar Cascade face detector
            self.face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        elif self.detection_type == "face_recognition":
            # Package face_detector doesn't require object initialization
            self.face_detector = None
        else: 
            raise ValueError("Choose one of the implemented models, ya cunt!")


    def detect_faces(self, frame):
        if self.detection_type == 'haarcascade':
            frame_pp = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            boxes = self.face_detector.detectMultiScale(frame_pp, scaleFactor=1.3, minNeighbors=3)
            
        elif self.detection_type == 'MTCNN':
            # No preprocessing needed
            boxes, _ = self.face_detector.detect(frame)
            
        elif self.detection_type == 'face_recognition':
            cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Package face_detector directly return boxes without object initialization
            boxes = face_recognition.face_locations(frame_pp, number_of_times_to_upsample=1)
        
        if (boxes is None) or (len(boxes) == 0):
            return [] 

        # Convert boxes to normalized format
        norm_boxes = self.get_norm_boxes(boxes)

        valid_boxes = []
        for box in norm_boxes: 
            box_rearranged = np.array([box[0],box[3], box[1], box[2]])
            face_landmark = face_recognition.face_landmarks(frame, [box_rearranged])[0]
            if FaceDetector.valid_landmarks(face_landmark):
                valid_boxes.append(box) 

        return valid_boxes

    def get_norm_boxes(self, boxes):
        """
        Normalize the bounding box coordinates from MTCNN to numpy indexing format.
        Output format: np.array(y_min, y_max, x_min, x_max)
        """
        normalized_box = []

        if self.detection_type == 'haarcascade':
            for box in boxes:
                x, y, w, h = box
                normalized_box.append([y, y+h, x, x+w])
            return np.array(normalized_box)    

        elif self.detection_type == "MTCNN":
            for box in boxes:
                x_min, y_min, x_max, y_max = box.astype(int)
                normalized_box.append([y_min, y_max, x_min, x_max])
            return np.array(normalized_box)

        elif self.detection_type =='face_recognition':
            for box in boxes:
                y_min, x_max, y_max, x_min = box
                normalized_box.append([y_min, y_max, x_min, x_max])
            return np.array(normalized_box)    
    
    @staticmethod
    def valid_landmarks(face_landmark):
        left_eye = face_landmark['left_eye']
        right_eye = face_landmark['right_eye']
        mouth = face_landmark['top_lip'] + face_landmark['bottom_lip']

        left_eye_center = np.mean(left_eye, axis=0)
        right_eye_center = np.mean(right_eye, axis=0)
        mouth_center = np.mean(mouth, axis=0)

        eye_distance = np.linalg.norm(right_eye_center - left_eye_center)
        mouth_to_eye_distance = np.linalg.norm(mouth_center - left_eye_center)

        return mouth_to_eye_distance > eye_distance * 0.8

class EmotionDetector:
    def __init__(self):
        self.emotion_detector = load_model("../models/emotion_model.hdf5", compile=False)
        self.emotion_categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    def predict(self, frame, box):

        gray_cropped_face = EmotionDetector.crop_face(box, frame)

        prob = self.emotion_detector.predict(gray_cropped_face)[0]  # check for underscore
        return prob
    
    @staticmethod
    def crop_face(box, frame):
        face = frame[box[0]:box[1], box[2]:box[3]]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face, (64, 64))
        face = face.astype('float32')/ 255
        face = np.expand_dims(face, axis=-1)
        face = np.expand_dims(face, axis=0)

        return face
    
class PersonIdentifier:
    def __init__(self, threshold = 0.6):
        self.threshold = threshold

    def assignID(self, frame, box)

In [36]:
#yt_link = 'https://www.youtube.com/watch?v=vtT78TfDfXU'                   # 1 Actor
#yt_link = 'https://www.youtube.com/watch?v=embYkODkzcs'                 # 7 basic emotions
#yt_link = 'https://www.youtube.com/watch?v=m70UInZKJjU'                    # Two persons
yt_link = 'https://www.youtube.com/watch?v=UECCHwh7bZE'

my_test = VideoProcessor(skip_frames=10, batch_size=10000)
my_face_detector = FaceDetector(detection_type='haarcascade')
my_emotion_detector = EmotionDetector()
my_person_identifier = PersonIdentifier()
emotions = []
frame_nr = 0
my_test.load_video_from_youtube(yt_link)
for idx, batch in enumerate(my_test.get_batches()):
    for frame in batch:
        face_boxes = my_face_detector.detect_faces(frame)
        
        if len(face_boxes) == 0:
            my_test.frames_without_faces += 1
            continue
        
        for box in face_boxes: 
            emotions.append((frame_nr, my_emotion_detector.predict(frame, box)))
            character_ID = my_person_identifier.assignID(frame, box)
        frame_nr +=1
            

1/1 [==============================] - 0s 363ms/step


2023-03-25 16:22:01.560797: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 35ms/step


KeyboardInterrupt: 

In [2]:
yt_link = 'https://www.youtube.com/watch?v=wo6K1GWEx84'

my_test = VideoProcessor(skip_frames=10)
my_face_detector = FaceDetector(detection_type='mtcnn')
my_test.load_video_from_youtube(yt_link)
for idx, batch in enumerate(my_test.get_batches()):
    boxes = my_face_detector.detect_faces(batch)
    

/Users/ben/neuefische/capstone/NeuralXpresso/.venv/lib/python3.9/site-packages/facenet_pytorch/models/utils/detect_face.py:250: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is "Min":
/Users/ben/neuefische/capstone/NeuralXpresso/.venv/lib/python3.9/site-packages/facenet_pytorch/models/utils/detect_face.py:250: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is "Min":
/Users/ben/neuefische/capstone/NeuralXpresso/.venv/lib/python3.9/site-packages/facenet_pytorch/models/utils/detect_face.py:250: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is "Min":


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (50,) + inhomogeneous part.

In [70]:
len(batchlist)

1

In [71]:
len(boxlist)

0

In [75]:
yt_link = 'https://www.youtube.com/watch?v=wo6K1GWEx84'
my_test = VideoProcessor(skip_frames=10)
my_face_detector = FaceDetector(detection_type='mtcnn')
#my_emotion_detector = EmotionDetector
batchlist = []
boxlist = []
my_test.load_video_from_youtube(yt_link)
for idx, batch in enumerate(my_test.get_batches()):
    print(f'{idx}: {len(batch)}')
    batchlist.append(batch)
    boxes = my_face_detector.detect_faces(batch)
    boxlist.append(boxes)

0: 50


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (50,) + inhomogeneous part.

In [3]:
batch.shape

(50, 720, 1280, 3)

In [128]:
face_detector_mtcnn = MTCNN()
a = face_detector_mtcnn.detect(batch[3:12])
print(a)

(array([[[ 800.64   ,  155.62999, 1065.035  ,  477.59427]],

       [[ 787.31104,  160.48291, 1047.8341 ,  486.15082]],

       [[ 736.2914 ,  159.95483,  996.07245,  482.7968 ]],

       [[ 724.2704 ,  159.92754,  975.6582 ,  487.37204]],

       [[ 715.5035 ,  159.72684,  956.12256,  473.42117]],

       [[ 205.15468,  180.71413,  391.8848 ,  428.30347]],

       [[ 213.19006,  178.01108,  399.1574 ,  423.1865 ]],

       [[ 227.02106,  182.68604,  408.90018,  420.32648]],

       [[ 761.26526,  181.73456, 1000.58075,  492.46167]]], dtype=float32), array([[0.9998952 ],
       [0.9997003 ],
       [0.99998546],
       [0.99999976],
       [0.9988213 ],
       [0.9996246 ],
       [0.99990594],
       [0.99990046],
       [0.99999857]], dtype=float32))


In [130]:
a[0].shape

(9, 1, 4)

In [4]:
import face_detection
# Initialize detector
detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)


/Users/ben/neuefische/capstone/NeuralXpresso/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ben/neuefische/capstone/NeuralXpresso/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:

# Getting detections
detections = detector.detect(batch)

In [5]:
detections = detector.batched_detect(batch)

KeyboardInterrupt: 

In [10]:
!pip install retina-face --no-deps

  Using cached retina_face-0.0.13-py3-none-any.whl (16 kB)


In [12]:
batch.shape

(50, 720, 1280, 3)

In [16]:
# Initialize detector
detector = cv2.FaceDetectorYN.create("/models/face_detection_yunet_2022mar.onnx", "", (320, 320))


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/dnn/src/onnx/onnx_importer.cpp:270: error: (-5:Bad argument) Can't read ONNX file: /models/face_detection_yunet_2022mar.onnx in function 'ONNXImporter'


In [ ]:
# Set input size
detector.setInputSize((1280, 720))
# Getting detections
detections = detector.detect(batch[0])

In [116]:
face_detector_mtcnn = MTCNN()
for i in range(50):
    a = face_detector_mtcnn.detect(batch[i])
    print(a)

(array([[ 213.83145,  219.39542,  415.32016,  483.84225],
       [ 834.7785 ,  208.32353, 1019.23663,  466.85956]], dtype=float32), array([0.99976796, 0.99993265], dtype=float32))
(array([[ 224.82489,  210.4985 ,  420.9738 ,  463.00748],
       [ 835.11536,  201.44376, 1015.41327,  461.78235]], dtype=float32), array([0.9999089 , 0.99991834], dtype=float32))
(array([[ 791.4241 ,  163.10724, 1054.3726 ,  494.60812]], dtype=float32), array([0.9998498], dtype=float32))
(array([[ 800.64   ,  155.63005, 1065.035  ,  477.5943 ]], dtype=float32), array([0.9998952], dtype=float32))
(array([[ 787.31104,  160.48288, 1047.8341 ,  486.1508 ]], dtype=float32), array([0.9997003], dtype=float32))
(array([[736.2914 , 159.95485, 996.07245, 482.7968 ]], dtype=float32), array([0.99998546], dtype=float32))
(array([[724.2704 , 159.92757, 975.6582 , 487.37204]], dtype=float32), array([0.99999976], dtype=float32))
(array([[715.5035 , 159.7268 , 956.12256, 473.4211 ]], dtype=float32), array([0.9988213], dtype=

In [115]:
a

(array([[ 224.82489,  210.4985 ,  420.9738 ,  463.00748],
        [ 835.11536,  201.44376, 1015.41327,  461.78235]], dtype=float32),
 array([0.9999089 , 0.99991834], dtype=float32))

In [89]:
batch[-1]

array([[[ 58,  28,  19],
        [ 58,  28,  19],
        [ 58,  28,  19],
        ...,
        [ 13,  90,   5],
        [ 16,  89,   5],
        [ 16,  89,   5]],

       [[ 58,  28,  19],
        [ 58,  28,  19],
        [ 56,  27,  18],
        ...,
        [ 13,  90,   5],
        [ 16,  89,   5],
        [ 16,  89,   5]],

       [[ 59,  30,  22],
        [ 59,  30,  22],
        [ 59,  30,  22],
        ...,
        [ 12,  90,   2],
        [ 15,  89,   2],
        [ 15,  89,   2]],

       ...,

       [[251, 221, 202],
        [251, 221, 202],
        [251, 221, 202],
        ...,
        [171, 157, 146],
        [171, 157, 146],
        [171, 157, 146]],

       [[251, 221, 202],
        [251, 221, 202],
        [251, 221, 202],
        ...,
        [172, 158, 147],
        [172, 158, 147],
        [172, 158, 147]],

       [[251, 221, 202],
        [251, 221, 202],
        [251, 221, 202],
        ...,
        [172, 158, 147],
        [172, 158, 147],
        [172, 158, 147]]

In [84]:
type(batch[0][0][0][0])

numpy.uint8

In [85]:
batch.shape

(50, 720, 1280, 3)

In [78]:
face_detector_mtcnn = MTCNN()
bboxes, probs = face_detector_mtcnn.detect(batch)
faces = []
for bbox in bboxes:
    xmin, ymin, xmax, ymax = bbox[0]
    box = [ymin, ymax, xmin, xmax]
    faces.append(box)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (50,) + inhomogeneous part.

In [77]:
batch

array([[[[ 31, 116,  40],
         [ 31, 116,  40],
         [ 31, 116,  40],
         ...,
         [242, 221, 194],
         [242, 221, 194],
         [242, 221, 194]],

        [[ 31, 116,  40],
         [ 31, 116,  40],
         [ 31, 116,  40],
         ...,
         [242, 221, 194],
         [242, 221, 194],
         [242, 221, 194]],

        [[ 33, 116,  40],
         [ 33, 116,  40],
         [ 33, 116,  40],
         ...,
         [242, 221, 194],
         [242, 221, 194],
         [242, 221, 194]],

        ...,

        [[211, 185, 168],
         [211, 185, 168],
         [210, 183, 167],
         ...,
         [ 31,   6,  65],
         [ 32,   9,  63],
         [ 33,  10,  65]],

        [[211, 185, 168],
         [211, 185, 168],
         [210, 183, 167],
         ...,
         [ 31,   8,  62],
         [ 31,   9,  61],
         [ 32,  10,  62]],

        [[211, 185, 168],
         [211, 185, 168],
         [210, 183, 167],
         ...,
         [ 31,   8,  62],
        

In [67]:
boxes.shape

(50, 1, 4)

In [76]:
boxlist

[]

In [39]:
boxes_list = []
for i, bbox in enumerate(boxes):
    xmin, ymin, xmax, ymax = bbox
    box = [ymin, ymax, xmin, xmax]
    boxes_list.append((i, box))

ValueError: not enough values to unpack (expected 4, got 1)

In [36]:
boxes_list

[(0, [62.388275, 314.8143, 534.76434, 730.4663])]

In [73]:
yt_link = 'https://www.youtube.com/watch?v=vtT78TfDfXU'
my_test = VideoProcessor(skip_frames=10)
my_face_detector = FaceDetector(detection_type='mtcnn')
my_test.load_video_from_youtube(yt_link)
my_test.total_frames


179

In [ ]:
# Initialize an empty numpy array to hold the frames
frames = np.empty((my_test.total_frames, my_test.height, my_test.width, 3), np.dtype('uint8'))

my_test.frames_read = 0
while True:
    ret, frame = self.video.read()
    
    if not ret:
    break

# Read the frames in batches and fill up the numpy array
for batch_start in range(0, self.frame_count, self.batch_size * self.skip_frames):
    batch_end = min(batch_start + (self.batch_size * self.skip_frames), self.frame_count)
    batch_index = 0

    for i in range(batch_start, batch_end):
        ret, frame = self.video.read()
        self.frames_read +=1
        if not ret:
            break

        if i % self.skip_frames == 0:
            frames[batch_index] = frame
            batch_index += 1

    # Resize the numpy array to fit the actual number of frames in the batch
    if batch_index < self.batch_size:
        frames = frames[:batch_index]

    # Yield the current batch of frames
    yield frames

# Release the video stream
self.video.release()

In [89]:
face_detector_hcc = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
gray = cv2.cvtColor(batch, cv2.COLOR_BGR2GRAY)
boxes = face_detector_hcc.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
boxes = [[y, y+h, x, x+w] for (x,y,w,h) in boxes]

error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<1, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 2, 5>, cv::impl::(anonymous namespace)::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<1, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [90]:
batch

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        ...,

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0,

In [61]:
my_test.__dict__

{'batch_size': 50,
 'skip_frames': 10,
 'yt_link': 'https://www.youtube.com/watch?v=vtT78TfDfXU',
 'video': < cv2.VideoCapture 0x14e376470>,
 'frame_count': 1785,
 'fps': 24,
 'height': 720,
 'width': 1280,
 'total_frames': 179,
 'num_batches': 4,
 'frames_read': 1785}

In [63]:
my_face_det = FaceDetector(detection_type="mtcnn")

In [62]:
batch[0].shape

(720, 1280, 3)

In [11]:
yt_link = 'https://www.youtube.com/watch?v=vtT78TfDfXU'                   # 1 Actor
#yt_link = 'https://www.youtube.com/watch?v=embYkODkzcs'                 # 7 basic emotions
#yt_link = 'https://www.youtube.com/watch?v=m70UInZKJjU'                    # Two persons

my_test = VideoProcessor(skip_frames=5)
my_test.get_video(yt_link)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
writer = cv2.VideoWriter('/Users/ben/neuefische/capstone/NeuralXpresso/notebooks/outputs/Output_video_3.mp4', fourcc, my_test.fps, (my_test.width, my_test.height))
counter = 0
frames = []
for idx, batch in enumerate(my_test.get_batches()):
    for frame in batch:
        frames.append(frame)
        writer.write(frame)
    counter +=1
writer.release()

In [ ]:
yt_link = 'https://www.youtube.com/watch?v=vtT78TfDfXU'                   # 1 Actor
#yt_link = 'https://www.youtube.com/watch?v=embYkODkzcs'                 # 7 basic emotions
#yt_link = 'https://www.youtube.com/watch?v=m70UInZKJjU'                    # Two persons

my_test = VideoProcessor(skip_frames=10)
my_face_detector = FaceDetector(detection_type='mtcnn')
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
writer = cv2.VideoWriter('/Users/ben/neuefische/capstone/NeuralXpresso/notebooks/outputs/Output_video_3.mp4', fourcc, my_test.fps, (my_test.width, my_test.height))
counter = 0
my_test.get_video(yt_link)
for idx, batch in enumerate(my_test.get_batches()):
    print(f'{idx}: {len(batch)}')
    boxes = my_face_detector.detect_faces(batch)
    break

In [ ]:
yt_link = 'https://www.youtube.com/watch?v=vtT78TfDfXU'                   # 1 Actor
#yt_link = 'https://www.youtube.com/watch?v=embYkODkzcs'                 # 7 basic emotions
#yt_link = 'https://www.youtube.com/watch?v=m70UInZKJjU'                    # Two persons

my_test = VideoProcessor(skip_frames=10)
my_face_detector = FaceDetector(detection_type='mtcnn')
#my_emotion_detector = EmotionDetector
batchlist = []
boxlist = []
my_test.get_video(yt_link)
for idx, batch in enumerate(my_test.get_batches()):
    print(f'{idx}: {len(batch)}')
    batchlist.append(batch)
    boxes = my_face_detector.detect_faces(batch)
    boxlist.append(boxes)
    #faces = extract_faces(batch, boxes, my_emotion_detector.input_shape)
    #emotions = my_emotion_detector.predict(faces)

In [102]:
len(frames)

357

In [ ]:
def output_video(video, filename):
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    return cv2.VideoWriter(filename, fourcc, 10, (width,height))

In [25]:
yt_video = YouTube(yt_link)
stream = yt_video.streams.get_highest_resolution()   
video =  cv2.VideoCapture(stream.url)


frame_count = 200
batch_size = 52

batches = []

for batch_start in range(0, frame_count, batch_size):
    batch_end = min(batch_start + batch_size, frame_count)
    frames = []



    # Read the frames in the current batch
    for i in range(batch_start, batch_end):
        counter = batch_start
        ret, frame = video.read()
        if not ret:
            break

        frames.append(frame)
        counter+=1

        if counter == batch_end:
            break
    batches.append(frames)

In [17]:
#     # Load the CNN face detector
face_detector = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/dnn/src/caffe/caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "deploy.prototxt" in function 'ReadProtoFromTextFile'


In [3]:
maha

NameError: name 'maha' is not defined

In [ ]:
for norm_box in norm_boxes: 
        if len(norm_boxes) == 0:
            frames_without_faces_counter += 1
            continue
        
        box = np.array([norm_box[0],norm_box[3], norm_box[1], norm_box[2]])
        face_landmark = face_recognition.face_landmarks(frame, [box])[0]
        if not valid_landmarks(face_landmark):
            continue

        gray_cropped_face = crop_face(norm_box, frame_pp)

        prob = emotions_probability(gray_cropped_face, emotion_classifier)
        emotions.append(prob)



        max_emotion, max_prob = np.argmax(prob), np.max(prob)
        emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        emotion_text = emotion_labels[max_emotion]



        face_encoding = face_recognition.face_encodings(frame, [box])[0]                  
        min_distance = float("inf")
        matched_individual_id = None

        for individual_id, individual_face_encoding in face_embeddings.items():
            distance = face_recognition.face_distance([individual_face_encoding], face_encoding)[0]
            if distance < 0.60: # Adjust this threshold value based on your requirements
                if distance < min_distance:
                    min_distance = distance
                    matched_individual_id = individual_id

        if matched_individual_id is None:
            individual_id_counter += 1
            matched_individual_id = individual_id_counter
            face_embeddings[individual_id_counter] = face_encoding

        y_min, y_max, x_min, x_max = norm_box
        
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(frame, f"Prob: {max_prob:.1%}", (x_min, y_max + 60), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 1)
        cv2.putText(frame, f"{emotion_text}", (x_min, y_max + 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 1)
        cv2.putText(frame, f"ID: {matched_individual_id}", (x_min, y_min -20), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 1)

                
        frame_info.append(
            (round(video.get(cv2.CAP_PROP_POS_MSEC) / 1000, 2),
                current_frame_nr
                )
        )    
